In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive


In [2]:
!pip install pyyaml==5.4.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 636 kB 28.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [5]:
%cd ./bias-bench
!python -m pip install -e .


/content/drive/MyDrive/bias-bench
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/drive/MyDrive/bias-bench
  Attempting uninstall: bias-bench
    Found existing installation: bias-bench 0.1.0
    Can't uninstall 'bias-bench'. No files were found to uninstall.
  Running setup.py develop for bias-bench


In [6]:
import json
import pandas as pd
from collections import defaultdict
res_dict_size = defaultdict(lambda: defaultdict(list))
model_sizes = ['gpt2', 'gpt2-medium', 'gpt2-large']

data_dir = './data/stereoset'

#model size experiment
for size in model_sizes:
    for run in range(5):
        !python experiments/stereoset.py --input_file "{data_dir}/gender_data_checklist_{run}.json" --model "GPT2LMHeadModel" --model_name_or_path "{size}" --seed 1 --experiment_name '{run}'
        !python experiments/stereoset_evaluation.py --predictions_file results/stereoset/stereoset_m-GPT2LMHeadModel_c-{size}_s-1_{run}.json   --output_file results/stereoset/results_{size}_{run}.json --data_file "{data_dir}/gender_data_checklist_{run}.json"
        !python experiments/stereoset_debias.py --input_file "{data_dir}/gender_data_checklist_{run}.json" --model "SelfDebiasGPT2LMHeadModel" --model_name_or_path "{size}" --seed 1 --experiment_name '{run}_debias'
        !python experiments/stereoset_evaluation.py --predictions_file results/stereoset/stereoset_m-SelfDebiasGPT2LMHeadModel_c-{size}_t-gender_s-1_{run}_debias.json   --output_file results/stereoset/results_{size}_{run}_debias.json --data_file "{data_dir}/gender_data_checklist_{run}.json"
        with open(f'./results/stereoset/results_{size}_{run}.json') as f:
            json_dict = json.load(f)
            res_dict_size['regular'][f'{size}'].append(float(list(json_dict.values())[0]['intrasentence']['implicit_gender']['SS Score']))
            
        with open(f'./results/stereoset/results_{size}_{run}_debias.json') as f:        
            json_dict = json.load(f)
            res_dict_size['debias'][f'{size}'].append(float(list(json_dict.values())[0]['intrasentence']['implicit_gender']['SS Score']))

Running StereoSet:
 - input_file: ../bias json/gender_data_checklist_0.json
 - persistent_dir: /content/drive/MyDrive/bias-bench
 - model: GPT2LMHeadModel
 - model_name_or_path: gpt2
 - batch_size: 1
 - seed: 1
 - experiment_id : stereoset_m-GPT2LMHeadModel_c-gpt2_s-1
 - experiment_name : 0
Downloading: 100% 665/665 [00:00<00:00, 1.18MB/s]
Downloading: 100% 523M/523M [00:09<00:00, 57.8MB/s]
Downloading: 100% 0.99M/0.99M [00:00<00:00, 59.4MB/s]
Downloading: 100% 446k/446k [00:00<00:00, 23.4MB/s]
Downloading: 100% 1.29M/1.29M [00:00<00:00, 28.8MB/s]
Using mask_token, but it is not set yet.
Evaluating intrasentence task.
100% 523/523 [00:09<00:00, 52.63it/s]
Evaluating StereoSet files:
 - predictions_file: results/stereoset/stereoset_m-GPT2LMHeadModel_c-gpt2_s-1_0.json
 - predictions_dir: None
 - output_file: results/stereoset/results_gpt2_0.json
 - data_file: ../bias json/gender_data_checklist_0.json
intrasentence
	implicit_gender
		Count: 1046.0
		LM Score: 0.0
		SS Score: 65.5831739961

In [11]:
import numpy as np
from collections import defaultdict
import pandas as pd
import plotly
pd.options.plotting.backend = "plotly"

res_dict_size_mean = defaultdict(dict)
for key in res_dict_size.keys():
    key2 = key.capitalize()
    for rc in res_dict_size[key].keys():
        if rc == 'gpt2':
            rc2 = 'GPT2-small'
        else:
            rc2=rc.replace('gpt', 'GPT')
        res_dict_size_mean[key2][rc2] = np.mean(res_dict_size[key][rc])
df =pd.DataFrame(res_dict_size_mean)


In [12]:
fig=df.plot.line(markers=True)#, color_discrete_sequence=['rgb(0,0,0)', '#AC1000'])
fig.update_layout(
    xaxis_title="Model Size",
    yaxis_title="Bias Score",
    legend_title=""
)

In [13]:
df['delta']=abs(50-df['Debias'])-abs(50-df['Regular'])
df

,Regular,Debias,delta
GPT2-small,63.824092,53.193117,-10.630975
GPT2-medium,60.650096,51.472275,-9.177820
GPT2-large,59.273423,50.095602,-9.177820
